## Exercise 4

**练习**：

构建指定的图，需要用到 **2 条条件边**。

输入：

```python
initial_state = AgentState(
    number1=10, operation="-", number2=5, number3=7, number4=2,
    operation2="+", finalNumber=0, finalNumber2=0
)
```

输出：

number1 - number2 = finalNumber

number3 + number4 = finalNumber2

In [24]:
from typing import TypedDict
from langgraph.graph import StateGraph,START,END

In [25]:
class AgentState(TypedDict):
    number1:int
    operation:str
    number2:int
    number3:int
    number4:int
    operation2:str
    finalNumber:int
    finalNumber2:int

In [26]:
def adder(state:AgentState) -> AgentState:
    """将两个数相加"""
    state["finalNumber"] = state["number1"] + state["number2"]
    return state

def subtractor(state:AgentState) -> AgentState:
    """将两个数相减"""
    state["finalNumber"] = state["number1"] - state["number2"]
    return state

def decide_next_node(state:AgentState) -> str:
    """路由"""
    if state["operation"] == "+":
        return "addition_operation"
    elif state["operation"] == "-":
        return "subtraction_operation"

def adder2(state:AgentState) -> AgentState:
    """将两个数相加"""
    state["finalNumber2"] = state["number3"] + state["number4"]
    return state

def subtractor2(state:AgentState) -> AgentState:
    """将两个数相减"""
    state["finalNumber2"] = state["number3"] - state["number4"]
    return state

def decide_next_node2(state:AgentState) -> str:
    """路由"""
    if state["operation2"] == "+":
        return "addition_operation2"
    elif state["operation2"] == "-":
        return "subtraction_operation2"

In [27]:
graph = StateGraph(AgentState)

graph.add_node("add_node",adder)
graph.add_node("subtract_node",subtractor)
graph.add_node("router",lambda state: state)
graph.add_node("add_node2",adder2)
graph.add_node("subtract_node2",subtractor2)
graph.add_node("router2",lambda state: state)

graph.add_edge(START,"router")
graph.add_conditional_edges(
    "router",
    decide_next_node,
    {
        "addition_operation":"add_node",
        "subtraction_operation":"subtract_node",
    }
)
graph.add_edge("add_node","router2")
graph.add_edge("subtract_node","router2")
graph.add_conditional_edges(
    "router2",
    decide_next_node2,
    {
        "addition_operation2":"add_node2",
        "subtraction_operation2":"subtract_node2",
    }
)
graph.add_edge("add_node2",END)
graph.add_edge("subtract_node2",END)

app = graph.compile()

In [28]:
initial_state = AgentState(
    number1=10, operation="-", number2=5, number3=7, number4=2,
    operation2="+", finalNumber=0, finalNumber2=0
)
answer = app.invoke(initial_state)

In [29]:
answer

{'number1': 10,
 'operation': '-',
 'number2': 5,
 'number3': 7,
 'number4': 2,
 'operation2': '+',
 'finalNumber': 5,
 'finalNumber2': 9}

In [30]:
answer["finalNumber"]

5

In [31]:
answer["finalNumber2"]

9